In [1]:
from transformers import pipeline

In [2]:
classifier = pipeline('sentiment-analysis')
classifier([
    "I've been waiting for EmKa Academy course my whole life",
    "I hate you brother..."
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9860089421272278},
 {'label': 'NEGATIVE', 'score': 0.9979550838470459}]

## What happens inside pipeline function ?

![All Stages](image-2.png)

![Alt Stage 1](image.png)

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for EmKa Academy course my whole life",
    "I hate you brother..."
]

inputs = tokenizer(raw_inputs, padding= True, truncation= True, return_tensors= "tf")
inputs

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 7861, 2912, 2914, 2607,
        2026, 2878, 2166,  102],
       [ 101, 1045, 5223, 2017, 2567, 1012, 1012, 1012,  102,    0,    0,
           0,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])>}

![Stage 2](image-1.png)

In [4]:
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

outputs = model(inputs)
print(outputs)

print(outputs.last_hidden_state.shape) # batch size, sequence length, hidden size

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(2, 15, 768), dtype=float32, numpy=
array([[[ 0.47508088,  0.12544139,  0.35949305, ..., -0.07024778,
          0.15129459,  0.17200321],
        [ 1.0680364 ,  0.8778791 , -0.12113761, ...,  0.10178837,
          0.5266532 , -0.00159217],
        [ 1.1748893 ,  0.06403016,  0.04088037, ...,  0.14178126,
         -0.3108806 , -0.561392  ],
        ...,
        [ 0.6119854 ,  0.32409376,  0.11707756, ..., -0.56740063,
          0.5678129 , -0.22134787],
        [ 0.8006039 ,  0.47104245,  0.15256916, ..., -0.42048514,
          0.25580367,  0.07919621],
        [ 0.81338173,  0.3965402 ,  0.09354299, ...,  0.25953385,
          0.0606913 , -0.31736967]],

       [[-0.34648955,  0.7238452 , -0.4847371 , ..., -0.06404313,
         -0.5665267 , -0.28546226],
        [-0.13217905,  1.1773334 , -0.44107527, ..., -0.13891427,
         -0.30249244,  0.133257  ],
        [-0.04977871,  1.1295692 , -0.43537778, ..., -0.3101096 ,
         -0.3

In [5]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(inputs)
print(outputs)

print(outputs.logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.1431468,  2.1121008],
       [ 3.3839605, -2.806392 ]], dtype=float32)>, hidden_states=None, attentions=None)
tf.Tensor(
[[-2.1431468  2.1121008]
 [ 3.3839605 -2.806392 ]], shape=(2, 2), dtype=float32)


![Stage 3](image-3.png)

In [6]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis = -1)
print(predictions)

tf.Tensor(
[[0.01399105 0.98600894]
 [0.9979551  0.00204491]], shape=(2, 2), dtype=float32)


In [7]:
model.config.id2label # # First sentence positive, second sentence negative.

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [8]:
# You know 3 stage inside the pipeline now.

In [9]:
# Done